In [1]:
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf 
import requests

from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D,Flatten,Dropout,MaxPool2D, BatchNormalization
from sklearn import metrics

In [2]:
df=pd.read_csv('../datasets/xss.csv', encoding='utf-8-sig')

df.head(5)

,Unnamed: 0,Sentence,Label
0,0,"<li><a href=""/wiki/File:Socrates.png"" class=""i...",0
1,1,"<tt onmouseover=""alert(1)"">test</tt>",1
2,2,"\t </span> <span class=""reference-text"">Steeri...",0
3,3,"\t </span> <span class=""reference-text""><cite ...",0
4,4,"\t </span>. <a href=""/wiki/Digital_object_iden...",0


In [3]:
df=df[df.columns[-2:]]

df.head(5)

,Sentence,Label
0,"<li><a href=""/wiki/File:Socrates.png"" class=""i...",0
1,"<tt onmouseover=""alert(1)"">test</tt>",1
2,"\t </span> <span class=""reference-text"">Steeri...",0
3,"\t </span> <span class=""reference-text""><cite ...",0
4,"\t </span>. <a href=""/wiki/Digital_object_iden...",0


In [4]:
sentences=df['Sentence'].values

In [5]:
def convert_to_ascii(sentence):
    sentence_ascii = []

    for i in sentence:
        if ord(i) < 8222:
            sentence_ascii.append(134)
        if ord(i) == 8221: 
            sentence_ascii.append(129)
        if ord(i) == 8220: 
            sentence_ascii.append(130)
        if ord(i) == 8216:
            sentence_ascii.append(131)
        if ord(i) == 8217: 
            sentence_ascii.append(132)
        if ord(i) == 8211: 
            sentence_ascii.append(133)

        if ord(i) <= 128:
            sentence_ascii.append(ord(i))

    if len(sentence_ascii) > 10000:
        sentence_ascii = sentence_ascii[:10000]
    else:
        sentence_ascii += [0] * (10000 - len(sentence_ascii))
    
    zer = np.array(sentence_ascii).reshape((100, 100))
    return zer

In [6]:
arr=np.zeros((len(sentences),100,100))

for i in range(len(sentences)):
    
    image=convert_to_ascii(sentences[i])

    x=np.asarray(image,dtype='float')
    image =  cv2.resize(x, dsize=(100,100), interpolation=cv2.INTER_CUBIC)
    image/=128

    arr[i]=image

In [7]:
data = arr.reshape(arr.shape[0], 100, 100, 1)

In [8]:
data.shape

(13686, 100, 100, 1)

In [9]:
y=df['Label'].values

In [10]:
trainX, testX, trainY, testY = train_test_split(data,y, test_size=0.2, random_state=42)
trainX, validX, trainY, validY = train_test_split(trainX,trainY, test_size=0.2, random_state=42)

In [11]:
model=tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64,(3,3), activation=tf.nn.relu, input_shape=(100,100,1)),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(256,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [12]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 64)        640       
                                                                 
 max_pooling2d (MaxPooling2  (None, 49, 49, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 23, 23, 128)       0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 21, 21, 256)       295168    
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 10, 10, 256)       0

In [13]:
batch_size = 128
num_epoch = 10

mc = ModelCheckpoint('../models/xss.h5', save_best_only=True, save_weights_only=True)
model_log = model.fit(trainX, trainY,
          batch_size=batch_size,
          epochs=num_epoch,
          verbose=1,
          validation_data=( validX,  validY),
          callbacks=[mc]
                     )

Epoch 1/10
69/69 [==============================] - 59s 855ms/step - loss: 0.4708 - accuracy: 0.7660 - val_loss: 0.3777 - val_accuracy: 0.8260
Epoch 2/10
69/69 [==============================] - 59s 859ms/step - loss: 0.3829 - accuracy: 0.8230 - val_loss: 0.3018 - val_accuracy: 0.8644
Epoch 3/10
69/69 [==============================] - 58s 847ms/step - loss: 0.1990 - accuracy: 0.9197 - val_loss: 0.1000 - val_accuracy: 0.9699
Epoch 4/10
69/69 [==============================] - 60s 865ms/step - loss: 0.0799 - accuracy: 0.9765 - val_loss: 0.1170 - val_accuracy: 0.9566
Epoch 5/10
69/69 [==============================] - 59s 860ms/step - loss: 0.0764 - accuracy: 0.9767 - val_loss: 0.0575 - val_accuracy: 0.9795
Epoch 6/10
69/69 [==============================] - 59s 850ms/step - loss: 0.0516 - accuracy: 0.9841 - val_loss: 0.0453 - val_accuracy: 0.9849
Epoch 7/10
69/69 [==============================] - 60s 865ms/step - loss: 0.0476 - accuracy: 0.9856 - val_loss: 0.0430 - val_accuracy: 0.9863

In [14]:
model.load_weights('../models/xss.h5')

y_prob = model.predict(testX)
threshold = 0.5
y_pred = (y_prob > threshold).astype(int)

clf_report = metrics.classification_report(testY, y_pred, digits=4)
cnf_matrix = metrics.confusion_matrix(testY, y_pred)
TN, FP, FN, TP = cnf_matrix.ravel()
TPR = TP / (TP + FN)
FPR = FP / (FP + TN)
FNR = FN / (FN + TP)

other_metrics = pd.DataFrame({'TPR': '%.4f' % TPR, 
                              'FPR': '%.4f' % FPR, 
                              'FNR': '%.4f' % FNR
                              }, index=[0]).to_string(col_space=9, index=False)
print(clf_report)
print(cnf_matrix)
print(other_metrics)

86/86 [==============================] - 5s 59ms/step
              precision    recall  f1-score   support

           0     0.9811    0.9873    0.9842      1260
           1     0.9891    0.9838    0.9864      1478

    accuracy                         0.9854      2738
   macro avg     0.9851    0.9855    0.9853      2738
weighted avg     0.9854    0.9854    0.9854      2738

[[1244   16]
 [  24 1454]]
      TPR       FPR       FNR
   0.9838    0.0127    0.0162


In [21]:
def xss_predict(model, sentence):
    preprocessed_sentence = convert_to_ascii(sentence)
    x = np.asarray(preprocessed_sentence, dtype='float')
    x = cv2.resize(x, dsize=(100, 100), interpolation=cv2.INTER_CUBIC)
    x /= 128
    x = np.expand_dims(x, axis=0) 
    x = np.expand_dims(x, axis=-1) 
    
    prediction = model.predict(x)
    
    return prediction[0][0]

def get_html_from_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.text
    except requests.exceptions.RequestException as e:
        print("Error while getting HTML", e)
        return None

test_url = "http://youtube.com"

your_sentence = get_html_from_url(test_url)
prediction = xss_predict(model, your_sentence)
if prediction > 0.5:
    print("Prediction: 1")
else:
    print("Prediction: 0")


1/1 [==============================] - 0s 15ms/step
Prediction: 0


In [22]:
def xss_predict(model, sentence):
    preprocessed_sentence = convert_to_ascii(sentence)
    x = np.asarray(preprocessed_sentence, dtype='float')
    x = cv2.resize(x, dsize=(100, 100), interpolation=cv2.INTER_CUBIC)
    x /= 128
    x = np.expand_dims(x, axis=0) 
    x = np.expand_dims(x, axis=-1) 
    
    prediction = model.predict(x)
    
    return prediction[0][0]

your_sentence = "<center draggable=""true"" ondrag=""alert(1)"">test</center>"
prediction = xss_predict(model, your_sentence)
if prediction > 0.5:
    print("Prediction: 1")
else:
    print("Prediction: 0")


1/1 [==============================] - 0s 18ms/step
Prediction: 1
